In [35]:
import pandas as pd
import numpy as np

### Insert all of your league settings and the path to your csv files with the data here.

In [36]:
pitching_sheet = '2018_Projected_P_Stats.csv'
C_sheet = '2018_Projected_C_Stats.csv'
firstbase_sheet = '2018_Projected_1B_Stats.csv'
secondbase_sheet = '2018_Projected_2B_Stats.csv'
SS_sheet = '2018_Projected_SS_Stats.csv'
thirdbase_sheet = '2018_Projected_3B_Stats.csv'
OF_sheet = '2018_Projected_OF_Stats.csv'
DH_sheet = '2018_Projected_DH_Stats.csv'


#INSERT SCORING FOR PITCHERS HERE
IP = 1.2
P_H = -.33
K = 1
P_BB = -.33 #include HBP
P_HR = 0
GS = 0
P_G = 0.5
ER = -2
W = -4
L = -4
SV = 3
P_SO = 1 #shutout
B = -1 #balk
PKO = 1 #pickoff, negligable
HD = 1 #hold
CG = 0.75 #complete game
QS = 1 #quality start
#################################

#INSERT SCORING FOR BATTERS HERE
B_G = 0
AB = 0
PA = 0
B_H = 0
single = 0.88
double = 1.2
triple = 1.55
B_HR = 1.98
R = 0
RBI = 0.75
B_BB = 0.69
SO = -0.2
HBP = 0.72
SB = 0.2
CS = -0.42
PO = 0
OFA = 0 #Outfield Assists
A = 0
E = -1
################################

#INSERT LEAGUE SETTINGS HERE
num_teams = 12
roster_size = 17 #exclude DL
dollars_per_team = 260
############################

In [37]:
pitchingDF = pd.read_csv(pitching_sheet)
C_DF = pd.read_csv(C_sheet)
firstbase_DF = pd.read_csv(firstbase_sheet)
secondbase_DF = pd.read_csv(secondbase_sheet)
thirdbase_DF = pd.read_csv(thirdbase_sheet)
SS_DF = pd.read_csv(SS_sheet)
OF_DF = pd.read_csv(OF_sheet)
DH_DF = pd.read_csv(DH_sheet)

In [38]:
def pitchingScoring(pitchingDF):
    
    pitchingDF['Points'] = (pitchingDF['IP']*IP) + (pitchingDF['H']*P_H) + (pitchingDF['SO']*K) + (pitchingDF['BB']*P_BB) + (pitchingDF['HR']*P_HR) + (pitchingDF['GS']*GS) + (pitchingDF['G']*P_G) + (pitchingDF['ER']*ER) + (pitchingDF['W']*W) + (pitchingDF['L']*L) + (pitchingDF['SV']*SV)
    pitchingDF['Pos'] = 'P'
    pitchingDF = pitchingDF.sort_values(by=['Points'], ascending=False)
    return pitchingDF

In [39]:
def battingScoring(battingDF, Pos):
    
    battingDF['Points'] = (battingDF['G']*B_G) + (battingDF['AB']*AB) + (battingDF['PA']*PA) + (battingDF['H']*B_H) + ((battingDF['H']-(battingDF['2B']+battingDF['3B']+battingDF['HR']))*single) + (battingDF['2B']*double) + (battingDF['3B']*triple) + (battingDF['HR']*B_HR) + (battingDF['R']*R) + (battingDF['RBI']*RBI) + (battingDF['BB']*B_BB) + (battingDF['SO']*SO) + (battingDF['HBP']*HBP) + (battingDF['SB']*SB) + (battingDF['CS']*CS) + ((battingDF['Fld']*PO)*3)
    
    #All values gotten from running SQL scripts by position since 2005 by game average
    if(Pos == 'C'):
        battingDF['Points'] += (((battingDF['G']*6.6)*PO)+((battingDF['G']*0.46)*A)+((battingDF['G']*0.05)*E))*((battingDF['Fld']+5)/10)
    if(Pos == '1B'):
        battingDF['Points'] += (((battingDF['G']*7.4)*PO)+((battingDF['G']*0.56)*A)+((battingDF['G']*0.05)*E))*((battingDF['Fld']+5)/10)*(battingDF['Fld']/10)
    if(Pos == '2B'):
        battingDF['Points'] += (((battingDF['G']*1.7)*PO)+((battingDF['G']*2.5)*A)+((battingDF['G']*0.07)*E))*((battingDF['Fld']+5)/10)
    if(Pos == '3B'):
        battingDF['Points'] += (((battingDF['G']*0.6)*PO)+((battingDF['G']*1.6)*A)+((battingDF['G']*0.1)*E))*((battingDF['Fld']+5)/10)
    if(Pos == 'SS'):
        battingDF['Points'] += (((battingDF['G']*1.3)*PO)+((battingDF['G']*2.6)*A)+((battingDF['G']*0.1)*E))*((battingDF['Fld']+5)/10)
    if(Pos == 'OF'):
        battingDF['Points'] += (((battingDF['G']*1.8)*PO)+((battingDF['G']*0.05)*OFA)+((battingDF['G']*0.03)*E))*((battingDF['Fld']+5)/10)
    
    battingDF['Pos'] = Pos
    battingDF = battingDF.sort_values(by=['Points'], ascending=False)
    return battingDF

In [40]:
tempDF = pd.concat([battingScoring(C_DF, 'C'), battingScoring(firstbase_DF, '1B'), battingScoring(secondbase_DF, '2B'), battingScoring(thirdbase_DF, '3B'), battingScoring(SS_DF, 'SS'), battingScoring(OF_DF, 'OF'), battingScoring(DH_DF, 'DH'), pitchingScoring(pitchingDF)], axis=0)

In [41]:
scoringDF = tempDF[['Name', 'Team', 'Pos', 'Points']]
scoringDF = scoringDF.sort_values(by=['Points'], ascending = False)
scoringDF = scoringDF.reset_index(drop=True)
scoringDF.drop(scoringDF.tail(7200).index,inplace=True)

In [42]:
def calculateValue(scoringDF):
    
    #INSERT LEAGUE SETTINGS HERE
    num_teams = 12
    roster_size = 17 #exclude DL
    dollars_per_team = 260
    ############################
    
    replacement_points = scoringDF['Points'][(num_teams*roster_size)+1]
    league_dollars = num_teams*dollars_per_team
    total_players = num_teams*roster_size
    
    total_points_above_replacement = 0
    for i in range(0,total_players):
        total_points_above_replacement += scoringDF['Points'][i]
    
    scoringDF['Value'] = (league_dollars * (scoringDF['Points']-replacement_points))/(total_points_above_replacement-(total_players*replacement_points))
    
    total_value_above_replacement = 0
    for i in range(0,total_players):
        total_value_above_replacement += scoringDF['Value'][i]
        
    scoringDF['Value'] = scoringDF['Value']*(league_dollars/total_value_above_replacement)
    
    return scoringDF
    
calculateValue(scoringDF)

,Name,Team,Pos,Points,Value
0,Max Scherzer,Nationals,P,339.720000,53.439866
1,Corey Kluber,Indians,P,337.470000,52.633551
2,Chris Sale,Red Sox,P,332.960000,51.017339
3,Mike Trout,Angels,OF,326.963600,48.868457
4,Clayton Kershaw,Dodgers,P,325.810000,48.455051
5,Giancarlo Stanton,Yankees,OF,306.292000,41.460543
6,Joey Votto,Reds,1B,304.374320,40.773319
7,Noah Syndergaard,Mets,P,301.960000,39.908119
8,Anthony Rizzo,Cubs,1B,300.842000,39.507470
9,Jacob deGrom,Mets,P,296.820000,38.066138
